# Systemverilog DPI

- 1. SystemVerilog DPI(Direct Programming Interface)?
  - SystemVerilog와 C 언어 사이의 인터페이스를 제공하는 기능
  - 쉽게 말하면, SystemVerilog 코드에서 C 함수를 호출하거나, 반대로 C 코드에서 SystemVerilog 함수나 태스크를 호출할 수 있게 해주는 다리 역할을 한다.
- 2. When to use?
  - 기존 C/C++ 라이브러리 재사용: 복잡한 알고리즘이나 모델이 이미 C로 구현되어 있다면, SystemVerilog에서 그대로 호출해 사용할 수 있다.
  - 성능 향상: 시뮬레이션 중 계산량이 많은 부분을 C로 처리하면 속도가 빨라질 수 있다.
  - 외부 시스템과 연동: 테스트벤치에서 외부 장치나 소프트웨어와 통신할 때 C를 통해 연동할 수 있다.
  - 테스트 자동화 및 유틸리티 함수: 파일 입출력, 문자열 처리 등 SystemVerilog에서 불편한 작업을 C로 처리할 수 있다.
- 3. How to use?
  - import 방식: SystemVerilog에서 C 함수를 호출
    - 예: import "DPI-C" function int my_c_func(int x);
  - export 방식: C에서 SystemVerilog 함수를 호출
    - 예: export "DPI-C" function void sv_func();
- 4. 동작 방식
  - Verilator를 사용할 때는 C++의 main() 함수가 시뮬레이션의 시작점이 되고, 그 안에서 Verilog로 작성된 하드웨어 모듈을 인스턴스화하고 호출하는 구조임
  - 이건 마치 C++이 "테스트벤치" 역할을 하면서 Verilog 모듈을 제어하는 방식이라고 보면 됨.
  - 정리:
    - (1) Verilog 모듈 작성 (예: top.v에 module top 정의)
    - (2) Verilator로 변환: Verilator가 Verilog 코드를 C++ 클래스로 변환 (예: Vtop 클래스 생성됨)
    - (3) C++에서 main() 함수 작성: 여기서 Vtop 객체를 만들고, eval()을 호출해서 시뮬레이션을 진행
    - (4) 시뮬레이션 실행: C++ 프로그램을 컴파일하고 실행하면 Verilog 모듈이 동작함
- 5. 핵심 포인트:
  - Verilator는 Verilog를 직접 시뮬레이션하지 않고, C++로 변환해서 실행하는 방식이다.
  - 따라서 C++이 "호스트" 역할을 하고, Verilog 모듈은 "게스트"처럼 동작하는 구조임.
  - eval() 함수는 Verilog의 시간 단위 동작을 평가하는 핵심 함수.

- SYstemVerilog DPI 사용 예:
  - SV → C 통신 확인: SystemVerilog가 C 함수(c_add_numbers)를 호출하고 결과를 받는다.
  - C → SV 통신 확인: C 함수(c_run_test)가 SystemVerilog 함수(sv_multiply)를 호출하고 결과를 받는다.

# Tool 설치

- Why Verilator?
  - DPI-C 완전 지원: SV에서 C 호출(import), C에서 SV 콜백(export) 둘 다 가능
  - 빠른 배치: 위 3파일만으로 개념을 바로 체득
  - 확장 용이
- Verilator 를 사용하기 위해서는
  - 작은 C++ 하니스(sim_main.cpp)가 필요
  - 하지만 DPI 함수 본체는 C로 그대로 쓰면 됨. (즉, C 함수 + C++ 메인 조합)

In [5]:
!apt-get update -y
!apt-get install -y verilator make g++ pkg-config

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,343 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubunt

#Example 1: SystemVerilog → C 호출 (일반적)
(Verilator 는 C++ 코드를 생성하므로 C 함수가 C++에서 링크될 수 있도록 해야 함)

- 파일 만들기 (SV + C + C++ 하니스)

In [20]:
%%writefile add_test.sv

import "DPI-C" function int add(int a, int b);

module add_test;

  // C 함수 import
  import "DPI-C" function int add(input int a, input int b);

  initial begin
    int result;
    result = add(7, 5);  // C 함수 호출
    $display("Result from C: %0d", result);
    $finish;
  end

endmodule


Overwriting add_test.sv


In [21]:
%%writefile add.c

#include <stdio.h>

/*
int add(int a, int b) {
    return a + b;
}
*/

extern "C" int add(int a, int b) {
    return a + b;
}


Overwriting add.c


In [32]:
%%writefile sim_main.cpp

#include "Vadd_test.h"      // Verilator가 생성한 클래스
#include "verilated.h"      // Verilator 지원 라이브러리

int main(int argc, char** argv) {        // Verilator로 변환된 C++ 시뮬레이터의 메인 함수
    Verilated::commandArgs(argc, argv);  // 명령줄 인자 처리 (Verilog Testbench에서 +옵션을 읽을 수 있게 해줌)

    Vadd_test* top = new Vadd_test;      // Verilator가 add_test.sv를 변환하면서 Vadd_test라는 C++ 클래스를 생성
                                         // Verilog의 최상위 모듈 인스턴스가  만들어짐.
                                         // 즉, RTL 시뮬레이션 객체가 C++에서 생성된 것.

    top->eval();                         // Verilator가 만든 함수로, Verilog의 시뮬레이션 스텝 (한 사이클)을 실행
                                         // 첫 호출에서는 Verilog의 initial 블록이 실행
                                         // 이후 반복적으로 eval()을 호출하면 always 블록, 클럭 이벤트 등이 진행
                                         // 여기서는 한 번만 호출했으므로 초기화 동작만 수행

    top->final();                        // Verilator가 제공하는 종료 함수
                                         // $finish 같은 Verilog 종료 이벤트를 처리하고, 내부적으로 trace 파일(.vcd)을
                                         // 닫거나 리소스를 정리

    delete top;                          // 메모리 해제
    return 0;
}


Overwriting sim_main.cpp


In [33]:
# Verilator 실행: SystemVerilog 코드를 C++ 시뮬레이터로 변환
# -Wall : 모든 경고 메시지 출력
# --cc add_test.sv : add_test.sv (SystemVerilog Testbench)를 C++ 코드로 변환.
# --exe sim_main2.cpp add.c : 변환된 Verilog 코드와 함께 sim_main.cpp(메인 실행 파일)과 add.c(DPI용 C 함수)를
#   링크해서 최종 실행 프로그램을 만들 준비.
# 결과: obj_dir/ 디렉토리에 Makefile(Vadd_test.mk)과 C++ 소스가 생성

!verilator -Wall --cc add_test.sv --exe sim_main.cpp add.c

# Make 실행: Verilator가 생성한 Makefile(Vadd_test.mk)을 이용해 실제 실행 파일을 빌드한다.
# -C obj_dir : obj_dir 디렉토리에서 빌드 수행.
# Vadd_test : 최종 실행 바이너리 이름.
# 결과: obj_dir/Vadd_test 실행 파일이 생성된다.

!make -C obj_dir -f Vadd_test.mk Vadd_test

# 시뮬레이션 실행: 앞에서 빌드된 실행 파일을 실행
# 동작과정:
#  1. Verilator가 변환한 C++ 시뮬레이터가 실행됨.
#  2. add_test.sv 안의 SystemVerilog Testbench가 구동됨.
#  3. Testbench가 DPI를 통해 add.c 안의 C 함수를 호출.
#  4. sim_main2.cpp는 시뮬레이션 메인 루프를 담당하며, 결과를 출력.

!./obj_dir/Vadd_test

make: Entering directory '/content/obj_dir'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow      -Os -c -o add.o ../add.c
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow      -Os -c -o sim_main.o ../sim_main.cpp
/usr/bin/perl /usr/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vadd_test.cpp Vadd_test___024unit.cpp Vadd_test__Dpi.cpp Vadd_test__Slow.cpp Vadd_test___024unit__Slow.cpp Vadd_test__Syms.cpp > Vadd_test__ALL.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/sh

```
[전체 흐름 요약]

SystemVerilog Testbench (add_test.sv)
        ↓ (Verilator 변환)
C++ 시뮬레이터 코드 + sim_main2.cpp + add.c
        ↓ (make 빌드)
실행 파일 obj_dir/Vadd_test
        ↓ (실행)
Verilog → C 함수 호출 (DPI)
결과 출력
```


# Example 2: C-> SystemVerilog 호출
- 특수한 경우(디버깅, HW-SW cosim)에만 주로 사용
  - 예: cosim_bfm_axi처럼 SW가 AXI 명령을 보내 RTL을 직접 제어할 때
- Verilator는 Verilog/SystemVerilog 모듈을 C++ 클래스로 변환할 때: "V + <Verilog 모듈 이름>" 으로 붙인다.

In [35]:
%%writefile top.v

module top;
  initial begin
    $display("Hello, Verilator! I am working...");
    $finish;
  end
endmodule

Overwriting top.v


In [36]:
%%writefile sim_main2.cpp

#include "Vtop.h"           // Verilator가 생성한 Verilog 모듈 클래스
#include "verilated.h"      // Verilator 시뮬레이션 지원 라이브러리

int main(int argc, char** argv) {
    Verilated::commandArgs(argc, argv);  // 시뮬레이터에 명령줄 인자 전달

    Vtop* top = new Vtop;   // Verilog 모듈 인스턴스 생성 (동적 할당, malloc())

    while (!Verilated::gotFinish()) {    // 시뮬레이션 종료 조건 체크
        top->eval();       // Verilog 모듈의 동작을 평가 (하드웨어 동작 시뮬레이션)
    }

    top->final();          // 시뮬레이션 종료 처리
    delete top;            // 메모리 해제 (free())
    return 0;
}

Overwriting sim_main2.cpp


In [37]:
!verilator -Wall --cc top.v --exe sim_main2.cpp
!make -C obj_dir -f Vtop.mk Vtop
!./obj_dir/Vtop

make: Entering directory '/content/obj_dir'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow      -Os -c -o sim_main2.o ../sim_main2.cpp
/usr/bin/perl /usr/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vtop.cpp Vtop__Slow.cpp Vtop__Syms.cpp > Vtop__ALL.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow      -Os -c -o Vtop__ALL.o Vtop__ALL.cpp
ar -cr Vtop__ALL.a Vtop__ALL.o
ranlib Vtop__ALL.a
g++    sim_main2.o verilated.o Vtop__ALL.a      -o Vtop
make: Leaving directory '/con

- 요약:
  - Verilog → C 호출 (import):
    - Verilog에서 C 함수를 호출
    - Verilator가 자동으로 DPI 연결 처리
    - 특별한 scope 설정 없이 eval()만으로 실행 가능
    - 예: import "DPI-C" function int add(int a, int b);
    - 간단하고 직관적 (C 함수는 독립적이므로 Verilog가 호출만 하면 됨)
  - C → Verilog 호출 (export)
    - C 코드에서 Verilog 태스크나 함수를 호출
    - Verilator는 context 키워드 미지원 → scope 수동 설정 필요
    - svSetScope()로 정확한 인스턴스 경로 지정해야 함
    - Verilog 태스크는 initial 블록 없이 정의되어 있어야 함
    - void 반환형은 task로 처리해야 함 (Verilator는 function void를 잘 못 다룸)
    - 복잡한 DPI 연결 (scope 설정, 인스턴스 경로, 태스크 구조 등 고려할 요소 많음)
  - Verilog는 하드웨어 기술 언어라서 외부 호출은 제한적이고, 내부에서 외부(C)로 호출하는 게 자연스러움
  - 반대로 C는 소프트웨어 언어라서 하드웨어의 상태나 인스턴스를 직접 다루려면 정확한 context와 scope가 필요함
  - Verilator는 시뮬레이터이기 때문에 하드웨어 인스턴스를 C에서 직접 제어하려면 명시적 설정이 필요함

-----------------------------------